<a href="https://colab.research.google.com/github/mohyunyang/My_Kaggle/blob/main/(Step2)Categorical_Feature_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기 

In [ ]:
! pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yhyunmo","key":"8fdb7bbf0193b24d0178986819fcf478"}'}

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle competitions download -c cat-in-the-dat
!ls
!unzip  cat-in-the-dat.zip

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
 33% 7.00M/21.2M [00:00<00:00, 73.2MB/s]
100% 21.2M/21.2M [00:00<00:00, 137MB/s] 
cat-in-the-dat.zip  kaggle.json  sample_data
Archive:  cat-in-the-dat.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# 2. 인코딩

In [ ]:
# 데이터 합치기
all_data = pd.concat([train,test]) 
all_data = all_data.drop('target', axis=1)
all_data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      500000 non-null  int64 
 1   bin_0   500000 non-null  int64 
 2   bin_1   500000 non-null  int64 
 3   bin_2   500000 non-null  int64 
 4   bin_3   500000 non-null  object
 5   bin_4   500000 non-null  object
 6   nom_0   500000 non-null  object
 7   nom_1   500000 non-null  object
 8   nom_2   500000 non-null  object
 9   nom_3   500000 non-null  object
 10  nom_4   500000 non-null  object
 11  nom_5   500000 non-null  object
 12  nom_6   500000 non-null  object
 13  nom_7   500000 non-null  object
 14  nom_8   500000 non-null  object
 15  nom_9   500000 non-null  object
 16  ord_0   500000 non-null  int64 
 17  ord_1   500000 non-null  object
 18  ord_2   500000 non-null  object
 19  ord_3   500000 non-null  object
 20  ord_4   500000 non-null  object
 21  ord_5   500000 non-null  object
 

In [ ]:
all_data.shape

(500000, 24)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)
all_data_encoded

<500000x516552 sparse matrix of type '<class 'numpy.float64'>'
	with 12000000 stored elements in Compressed Sparse Row format>

데이터 나누기

In [ ]:
num_train = len(train)

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size = 0.1,     # 10% 로 검증데이터 배분
                                                      stratify = y,        # 타깃값을 공정하게 배분
                                                      random_state = 10)



# 3. 모델훈련

max_iter : 모델의 회귀 계수를 업데이트하는 반복 횟수, 모델 훈련 시 회귀 계수를 업데이트하면서 훈련하는데, 이때 업데이트를 몇 번 할지를 정함

In [ ]:
from sklearn.utils.extmath import log_logistic
from sklearn.linear_model import LogisticRegression

logi_model = LogisticRegression(max_iter= 1000, random_state= 42) # 모델생성
logi_model.fit(X_train, y_train) # 모델훈련

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
logi_model.predict_proba(X_valid)   # 양성일 확률 예측

array([[0.23057724, 0.76942276],
       [0.91580546, 0.08419454],
       [0.84242849, 0.15757151],
       ...,
       [0.25273963, 0.74726037],
       [0.50728863, 0.49271137],
       [0.95770316, 0.04229684]])

In [ ]:
logi_model.predict(X_valid) # 타깃값 예측

array([1, 0, 0, ..., 1, 0, 0])

In [ ]:
# 검증 데이터를 활용한 타깃 예측,
y_valid_preds = logi_model.predict_proba(X_valid)[:, 1] # 변수에는 검증데이터 타깃값이 1일 확률을 저장

In [ ]:
y_valid_preds[:5]

array([0.76942276, 0.08419454, 0.15757151, 0.13839104, 0.85154103])

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)  # 타깃값을 예측값으로 나눔

print(f'검증데이터 ROC AUC : {roc_auc:.4f}')

검증데이터 ROC AUC : 0.7976


# 4. 예측 및 결과 제출

In [ ]:
# 타깃값이 1일 확률 예측
y_preds = logi_model.predict_proba(X_test)[:, 1]
print(y_preds[:5])

[0.31121091 0.70078041 0.06857948 0.44621462 0.89183416]


In [ ]:
# 제출파일생성
submission['target'] = y_preds
submission.to_csv('CFE2_submission.csv', index=False)

In [ ]:
submission.head()

,id,target
0,300000,0.311211
1,300001,0.700780
2,300002,0.068579
3,300003,0.446215
4,300004,0.891834
